In [67]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from torch.utils.data import TensorDataset
# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

In [68]:
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
boston_raw = pd.read_csv('../data/housing.csv', header=None, delimiter=r"\s+", names=column_names)

boston = boston_raw.copy()

In [69]:
train_boston = boston.sample(frac=0.8, random_state=0)
test_boston = boston.drop(train_boston.index)

In [70]:
train_stats = train_boston.describe()
train_stats.pop("MEDV")
train_stats = train_stats.transpose()
# train_stats

In [71]:
train_targets = train_boston.pop('MEDV')
test_targets = test_boston.pop('MEDV')


In [72]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    return torch.from_numpy(df.values).float().to(device)

Using cpu device


In [73]:
# The tf.keras.layers.Normalization is a clean and simple way to add feature normalization into your model.
# It is true but since version 2.6.0
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']

# print((train_boston),(train_targets))
normed_train_dataset = TensorDataset(df_to_tensor(train_boston), df_to_tensor(train_targets))
normed_test_dataset = TensorDataset(df_to_tensor(test_boston), df_to_tensor(test_targets))

train_dataloader = DataLoader(normed_train_dataset,batch_size=32)
test_dataloader = DataLoader(normed_test_dataset,batch_size=32)
# print(train_dataloader)
# for data in train_dataloader.dataset:
#   print(data)

`Learn the Basics <intro.html>`_ ||
`Quickstart <quickstart_tutorial.html>`_ ||
`Tensors <tensorqs_tutorial.html>`_ ||
`Datasets & DataLoaders <data_tutorial.html>`_ ||
`Transforms <transforms_tutorial.html>`_ ||
**Build Model** ||
`Autograd <autogradqs_tutorial.html>`_ ||
`Optimization <optimization_tutorial.html>`_ ||
`Save & Load Model <saveloadrun_tutorial.html>`_

Build the Neural Network
===================

Neural networks comprise of layers/modules that perform operations on data.
The `torch.nn <https://pytorch.org/docs/stable/nn.html>`_ namespace provides all the building blocks you need to
build your own neural network. Every module in PyTorch subclasses the `nn.Module <https://pytorch.org/docs/stable/generated/torch.nn.Module.html>`_.
A neural network is a module itself that consists of other modules (layers). This nested structure allows for
building and managing complex architectures easily.

In the following sections, we'll build a neural network to classify images in the FashionMNIST dataset.

In [74]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Get Device for Training
-----------------------
We want to be able to train our model on a hardware accelerator like the GPU,
if it is available. Let's check to see if
`torch.cuda <https://pytorch.org/docs/stable/notes/cuda.html>`_ is available, else we
continue to use the CPU.


Define the Class
-------------------------
We define our neural network by subclassing ``nn.Module``, and
initialize the neural network layers in ``__init__``. Every ``nn.Module`` subclass implements
the operations on input data in the ``forward`` method.


In [75]:
learning_rate = 0.001
batch_size = 2
epochs = 13
print(learning_rate)

0.001


In [76]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(13, 5),
            nn.ReLU()
            )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of ``NeuralNetwork``, and move it to the ``device``, and print
its structure.

In [77]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=13, out_features=5, bias=True)
    (1): ReLU()
  )
)


In [78]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [79]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        # print(X,y)
        pred = model(X)
        loss = loss_fn(pred, y)        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    plt.clf()

    # Get predictions

    # Plot true data
    plt.plot(train_boston, train_targets, 'go', label='True data', alpha=0.5)

    # Plot predictions
    plt.plot(train_boston, pred, '--', label='Predictions', alpha=0.5)

    # Legend and plot
    plt.legend(loc='best')
    plt.show()

In [80]:
epochs = 10
for t in range(epochs):
    # print(f"Epoch {t+1}\n-------------------------------")
    # print(len(train_dataloader.dataset))
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")


/usr/local/lib/python3.9/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (5) must match the size of tensor b (32) at non-singleton dimension 1